In [62]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 21: Fractal Art ---</h2><p>You find a program trying to generate some art. It uses a strange process that involves <span title="This technique is also often used on TV.">repeatedly enhancing</span> the detail of an image through a set of rules.</p>
<p>The image consists of a two-dimensional square grid of pixels that are either on (<code>#</code>) or off (<code>.</code>). The program always begins with this pattern:</p>
<pre><code>.#.
..#
###
</code></pre>
<p>Because the pattern is both <code>3</code> pixels wide and <code>3</code> pixels tall, it is said to have a <em>size</em> of <code>3</code>.</p>
<p>Then, the program repeats the following process:</p>
<ul>
<li>If the size is evenly divisible by <code>2</code>, break the pixels up into <code>2x2</code> squares, and convert each <code>2x2</code> square into a <code>3x3</code> square by following the corresponding <em>enhancement rule</em>.</li>
<li>Otherwise, the size is evenly divisible by <code>3</code>; break the pixels up into <code>3x3</code> squares, and convert each <code>3x3</code> square into a <code>4x4</code> square by following the corresponding <em>enhancement rule</em>.</li>
</ul>
<p>Because each square of pixels is replaced by a larger one, the image gains pixels and so its <em>size</em> increases.</p>
<p>The artist's book of enhancement rules is nearby (your puzzle input); however, it seems to be missing rules.  The artist explains that sometimes, one must <em>rotate</em> or <em>flip</em> the input pattern to find a match. (Never rotate or flip the output pattern, though.) Each pattern is written concisely: rows are listed as single units, ordered top-down, and separated by slashes. For example, the following rules correspond to the adjacent patterns:</p>
<pre><code>../.#  =  ..
          .#

                .#.

.#./..#/### = ..# ###

                        #..#

#..#/..../#..#/.##. = ....
#..#
.##.
</code></pre>

<p>When searching for a rule to use, rotate and flip the pattern as necessary.  For example, all of the following patterns match the same rule:</p>
<pre><code>.#.   .#.   #..   ###
..#   #..   #.#   ..#
###   ###   ##.   .#.
</code></pre>
<p>Suppose the book contained the following two rules:</p>
<pre><code>../.# =&gt; ##./#../...
.#./..#/### =&gt; #..#/..../..../#..#
</code></pre>
<p>As before, the program begins with this pattern:</p>
<pre><code>.#.
..#
###
</code></pre>
<p>The size of the grid (<code>3</code>) is not divisible by <code>2</code>, but it is divisible by <code>3</code>. It divides evenly into a single square; the square matches the second rule, which produces:</p>
<pre><code>#..#
....
....
#..#
</code></pre>
<p>The size of this enhanced grid (<code>4</code>) is evenly divisible by <code>2</code>, so that rule is used. It divides evenly into four squares:</p>
<pre><code>#.|.#
..|..
--+--
..|..
#.|.#
</code></pre>
<p>Each of these squares matches the same rule (<code>../.# =&gt; ##./#../...</code>), three of which require some flipping and rotation to line up with the rule. The output for the rule is the same in all four cases:</p>
<pre><code>##.|##.
#..|#..
...|...
---+---
##.|##.
#..|#..
...|...
</code></pre>
<p>Finally, the squares are joined into a new grid:</p>
<pre><code>##.##.
#..#..
......
##.##.
#..#..
......
</code></pre>
<p>Thus, after <code>2</code> iterations, the grid contains <code>12</code> pixels that are <em>on</em>.</p>
<p><em>How many pixels stay on</em> after <code>5</code> iterations?</p>
</article>


In [63]:
from numpy import flip, int8


start = """
.#.
..#
###
"""

rules = """
../.# => ##./#../...
.#./..#/### => #..#/..../..../#..#                  
"""


def parse(s: str) -> np.ndarray[int8]:
    return np.array(
        [[c == "#" for c in l] for l in s.strip().splitlines()], dtype="int"
    )


def _hash(m: np.ndarray[int8]) -> int:
    return hash(bytes(m))


def parse_rules(rules: str) -> dict[int, np.ndarray[int8]]:
    rules_map = {}
    for rule in rules.strip().splitlines():
        k, v = (parse(r.replace("/", "\n")) for r in rule.split(" => "))
        rules_map[_hash(k)] = v
    return rules_map


def get_rule(
    m: np.ndarray[int8], rules: dict[int, np.ndarray[int8]]
) -> np.ndarray[int8]:
    for _ in range(3):
        i = _hash(m)
        if i in rules:
            return rules[i]
        i = _hash(np.flip(m, 0))
        if i in rules:
            return rules[i]
        i = _hash(np.flip(m, 1))
        if i in rules:
            return rules[i]
        m = np.rot90(m)


def generate(start: str, rules: str, cycles: int = 2) -> np.ndarray[int8]:
    m = parse(start)
    rules = parse_rules(rules)
    for _ in range(cycles):
        n = m.shape[0]
        b = 2 if n % 2 == 0 else 3
        m = np.block(
            [
                [get_rule(m[y - b : y, x - b : x], rules) for x in range(b, n + 1, b)]
                for y in range(b, n + 1, b)
            ]
        )

    return m


def count_pixels(start: str, rules: str, cycles=2) -> int:
    m = generate(start, rules, cycles)
    return m.sum()


print(f"Example {count_pixels(start, rules, cycles=2)} should be 12")

Example 12 should be 12


In [64]:
with open("../input/day21.txt") as f:
    puzzle = f.read()


print(f"Part I: {count_pixels(start, puzzle, cycles=5)}")

Part I: 117


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>117</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p><em>How many pixels stay on</em> after <code>18</code> iterations?</p>
</article>

</main>


In [65]:
print(f"Part II: {count_pixels(start, puzzle, cycles=18)}")

Part II: 2026963


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>2026963</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
